In [1]:
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras.datasets import imdb
import math
from keras.preprocessing  import sequence
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense
import re

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
def decode(msg,rev_index):
    return " ".join( [reverse_index.get(i, "#") for i in msg ])

In [68]:
def encode(msg,index):
    return np.array([index.get(i) for i in msg.split(" ")])

In [16]:
def preprocess(sent):
    return re.sub(r'[^\w\s]','',sent).lower()
    

In [69]:
def analyze(sent,model):
    sent = preprocess(sent)
    e = encode(sent,index)
    e[e>10000] = 7
    e_pad = sequence.pad_sequences([e], maxlen=max_review_length)
    return model.predict(e_pad)

In [2]:
top_words = 10000 
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17465344/17464789 [==============================] - 12s 1us/step


In [3]:
max_review_length = 5000

In [4]:
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [5]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()]) 

1646592/1641221 [==============================] - 6s 3us/step


In [6]:
embedding_vector_length = 32 
model = Sequential() 
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length)) 
model.add(LSTM(100)) 
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy']) 
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5000, 32)          320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 373,301
Trainable params: 373,301
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=3, batch_size=64) 

/usr/local/lib/python3.5/site-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 3222s - loss: 0.4784 - acc: 0.7600 - val_loss: 0.5359 - val_acc: 0.7320
Epoch 2/3
25000/25000 [==============================] - 3200s - loss: 0.2880 - acc: 0.8834 - val_loss: 0.3223 - val_acc: 0.8742
Epoch 3/3
25000/25000 [==============================] - 3192s - loss: 0.2079 - acc: 0.9220 - val_loss: 0.3361 - val_acc: 0.8725


In [ ]:
model.save("sentiment.h5")

In [7]:
model = model.load_weights("sentiment.h5")

https://www.liip.ch/en/blog/sentiment-detection-with-keras-word-embeddings-and-lstm-deep-learning-networks

In [11]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()])

In [105]:
sen = "the movie was not bad but I loved the story"
analyze(sen,model)

array([[0.30247867]], dtype=float32)

In [71]:
model.evaluate(X_test,y_test)

25000/25000 [==============================] - 1373s 55ms/step


[0.3360979736185074, 0.87248]